## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime
import numpy as np


In [2]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [3]:
def submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid, name, check_sheet, ars_url='https://ars-dev.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    dict3 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                    if name in list(check_sheet.Workflow):
                        print(name)
                        dfy = check_sheet[check_sheet['Workflow']== name]

                        dfy.reset_index(drop=True)

                        for index,curie_id in enumerate(dfy.Curie):
                            print(index,curie_id)
                            node_num = dfy.iloc[index][3]
                            query_id = curie_id
                            if np.isnan(dfy.iloc[index][2]) == True:
                                len_check = len(child_response['fields']['data']['message']['results'])
                            else:
                                len_check = dfy.iloc[index][2]

                            #print(node_num, query_id, len_check)

                            locs = []
                            for x, val in enumerate(child_response['fields']['data']['message']['results']):
                                #print(val)

                                if x < len_check:

                                    if query_id in val['node_bindings'][node_num][0]['id']:
                                        locs.append(x)
                            if not locs:
                                check_result = f'False'
                                print(check_result)
                                #pass
                            else:
                                check_result = f'True'
                                print('curie id:', query_id, ': INCLUDED at postion N ==', locs, 'on', node_num)

                            dict3[curie_id] = check_result    

                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        
                                        
                                        value_att = tx['value']
                        
                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))
                                        
                                        
                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code) + ' ' + str(dict3)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [4]:
check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

In [5]:
check_sheet

,Workflow,Curie,N (size of list of results),Query node ID,Include/Exclude,Unnamed: 5,Unnamed: 6
0,A.0_RHOBTB2_direct.json,PUBCHEM.COMPOUND:2950270,10.0,n1,Include,NaN,Note: this example row means that for workflow...
1,B.1_DILI-three-hop-interacts-with.json,CHEBI:41879,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
2,B.1_DILI-three-hop-interacts-with.json,MESH:D000077185,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
3,B.1_DILI-three-hop-interacts-with.json,MESH:D000077385,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
4,B.1_DILI-three-hop-interacts-with.json,MESH:D003474,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
5,B.1_DILI-three-hop-interacts-with.json,RXNORM:40068,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
6,B.1_DILI-three-hop-interacts-with.json,PUBCHEM.COMPOUND:5877,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
7,B.1_DILI-three-hop-interacts-with.json,PUBCHEM.COMPOUND:5755,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
8,C.1a_SmallMolecule_real_world_evidence_MultScl...,MONDO:0005301,100.0,n00,include,NaN,NaN
9,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallM...,CHEBI:45783,100.0,n3,include,NaN,NaN


In [6]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            if (name == 'C.1_Template_SmallMolecule_real_world_evidence_Disease.json') or (name == 'C.2_Template_Drug_Disease_GeneSet_interacts_with_SmallMolecule.json') or (name == 'C.3_Template_Disease_related_to_Drug.json'):
                pass
            else:
                file_read = path.join(root, name)
                dir_name = (os.path.splitext(os.path.basename(root))[0])
                print(file_read)

                filename = (os.path.splitext(os.path.basename(file_read))[0])
                print(filename)
                with open(file_read,'r') as inf:
                    query = json.load(inf)

                    kcresult = submit_to_ars(query)

                    sleep(400)

                    result_status = retrieve_ars_results(kcresult, name, check_sheet)


                    dict_workflows[filename] = kcresult

                    sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.0_RHOBTB2_direct.json
A.0_RHOBTB2_direct
https://arax.ncats.io/?source=ARS&id=5b0948b3-a866-451b-825d-01215750f101
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 0
Done
ara-bte Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.0_RHOBTB2_direct_inverse.json
A.0_RHOBTB2_direct_inverse
https://arax.ncats.io/?source=ARS&id=0ee08bf6-c7de-4dd1-bf2c-f586c3755d67
Done
Done
ara-aragorn Done 4
Done
ara-arax Done 4
Done
ara-bte Done 4
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2_RHOBTB2_twohop.json
A.2_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=e32bbb32-ee89-41cc-a45e-44fcc405b8f0
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 500
Done
ara-bte Done 1860
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2a_expanded_RHOBTB2_twohop.json
A.2a_expanded_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=326ac6d2-d094-4a39-ab8b-2daf85985

https://arax.ncats.io/?source=ARS&id=2c6fa1b1-fd74-4cca-afa8-f4046aaa15aa
Done
Done
ara-arax ARS Error 0
Done
ara-aragorn ARS Error 0
Done
D.6_metformin-ferritin.json
0 UniProtKB:P54646
ara-bte ARS Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowD/D.2_ssri-heart-disease.json
D.2_ssri-heart-disease
https://arax.ncats.io/?source=ARS&id=6113e273-8ac0-4fe3-a046-633857e50219
Done
Done
D.2_ssri-heart-disease.json
0 NCBIGene:3988
curie id: NCBIGene:3988 : INCLUDED at postion N == [48, 50, 127, 133, 329, 331, 334, 335, 355, 358, 373, 376, 391, 393, 424, 425, 450, 613, 614, 853, 854, 1155, 1156, 1227, 1235, 1240, 1245, 1250, 1256, 1261, 1266, 1270, 1439, 1441, 1442, 1444, 1445, 1446, 1453, 1456, 1553, 1555, 1746, 1747, 1761, 1906, 1953, 1957, 1995, 2093, 2098, 2107, 2114, 2131, 2134, 2197, 2203, 2424, 2527, 2528, 2529, 2530, 2533, 2534, 2537, 2626, 2656, 2657, 2662, 2917, 2963, 2966, 2969, 2972, 3112, 3260, 3261, 3301, 3302, 3303, 3308, 3365, 3496, 3530, 3531, 3565, 

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [7]:
dict_workflows

{'A.0_RHOBTB2_direct': '884e4d14-f370-4241-90e5-8dc9bd836e1f',
 'A.0_RHOBTB2_direct_inverse': '0ee08bf6-c7de-4dd1-bf2c-f586c3755d67',
 'A.2_RHOBTB2_twohop': 'e32bbb32-ee89-41cc-a45e-44fcc405b8f0',
 'A.2a_expanded_RHOBTB2_twohop': '326ac6d2-d094-4a39-ab8b-2daf859857b2',
 'A.3_KCNMA1': '95252e21-e1d6-4f17-bb61-10f6a8eff521',
 'A.8_EGFR_simple': '6558b545-8a8b-418e-b4ab-b26df51916ba',
 'A.Multiomics_BigGIM_DR_KP_RHOBTB2': '9faa3556-d1e1-414b-887c-3ef377fe522a',
 'A.Multiomics_BigGIM_DR_KP_EGFR': '4a0d9947-66d3-4df5-bf93-9621779d38bd',
 'A.9_EGFR_advanced': '550a0683-97d9-45ea-a676-bcf53460bfd2',
 'A.1_RHOBTB2': '121e30f6-449f-42d1-8180-8f11a8db968e',
 'A.2a_RHOBTB2_twohop': '012e93f1-7abc-40a1-9371-3164bf7fb56e',
 'A.2_RHOBTB2_twohop_constrained': 'd604d4e9-f3d4-4cab-ab3e-c0361592217a',
 'A.2a_expanded_RHOBTB2_twohop_constrained': '4844d5f8-7abf-46ab-a0e4-006506553199',
 'B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol': 'a30085be-58c5-42f0-ac66-848ac88c31a9',
 'B.2f_DILI-fourth

In [8]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    name = keys + '.json'
    print(name, val)
    

    result_status = retrieve_ars_results(val, name, check_sheet)
    
    workflow_result_messages[keys] = result_status

A.0_RHOBTB2_direct.json 884e4d14-f370-4241-90e5-8dc9bd836e1f
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 0
Done
ara-bte Done 0
A.0_RHOBTB2_direct_inverse.json 0ee08bf6-c7de-4dd1-bf2c-f586c3755d67
Done
Done
ara-aragorn Done 4
Done
ara-arax Done 4
Done
ara-bte Done 4
A.2_RHOBTB2_twohop.json e32bbb32-ee89-41cc-a45e-44fcc405b8f0
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 500
Done
ara-bte Done 1860
A.2a_expanded_RHOBTB2_twohop.json 326ac6d2-d094-4a39-ab8b-2daf859857b2
Done
Done
ara-aragorn Done 1894
Done
ara-arax Done 35
Done
ara-bte Done 37
A.3_KCNMA1.json 95252e21-e1d6-4f17-bb61-10f6a8eff521
Done
Done
ara-aragorn Done 931
Done
ara-arax Done 182
Done
ara-bte Done 133
A.8_EGFR_simple.json 6558b545-8a8b-418e-b4ab-b26df51916ba
Done
Done
ara-aragorn Done 1991
Done
ara-arax Done 459
Done
ara-bte Done 157
A.Multiomics_BigGIM_DR_KP_RHOBTB2.json 9faa3556-d1e1-414b-887c-3ef377fe522a
Done
Done
ara-aragorn Done 4
Done
ara-arax Done 3
Done
ara-bte Done 4
A.Multiomics_BigGIM_DR_KP_EGFR.js

ara-aragorn Done 0
Done
ara-arax Done 1
Done
ara-bte Done 0
C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule.json 2f9cc878-513b-42d5-a31c-d82a9b7a60d0
Done
Done
ara-aragorn Done 0
Done
C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule.json
0 CHEBI:45783
False
ara-arax Done 100
Done
ara-bte ARS Error 0


### Creating dataframe for workflows with PK

<br>

In [ ]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)

for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k][0].items():
        #print(key, value)
#         if key.startswith('kp-'):
#             key_mod = key.replace('kp-','')
#         else:
#             key_mod = key
        
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

In [ ]:
df

In [ ]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

In [ ]:
df

In [ ]:
df.replace('{}','',regex=True,inplace=True)

### Creating second table with edge attribute source

<br>

In [ ]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    count = 0
    
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

In [ ]:
final_dict2

In [ ]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [ ]:
final_dictassemble

In [ ]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [ ]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [ ]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [ ]:
df2test = pd.DataFrame(df2test.unstack().T)

In [ ]:
df2test

In [ ]:
#df2test.drop([''], axis=0, inplace=True)

In [ ]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [ ]:
#df2.replace([], 'None', regex=True,inplace=True)

In [ ]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [ ]:
df2test = df2test.rename_axis(None)

In [ ]:
df2test.columns.name = None

In [ ]:
df2test.fillna('', inplace=True)

In [ ]:
df2test

In [ ]:
infores_catalog = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/InfoRes Catalog - Translator InfoRes Catalog.csv", header=1)

In [ ]:
infores_catalog = infores_catalog[['id', 'name','translator category','has contributor']]
infores_catalog = infores_catalog[:335]
dict_map = {}
for i in df2test.index.values:
    if i in infores_catalog['id'].values:
        indices = infores_catalog[infores_catalog['id']==i].index[0]
        if pd.notnull(infores_catalog.iloc[indices]['has contributor']):
            dict_map[i] = infores_catalog.iloc[indices]['translator category']
        else:
            dict_map[i] = 'External Source'
    else:
        dict_map[i] = 'Illegal value'

        
        
        
        
        
        
df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog']=df2test.index.map(dict_map)
df2test['sort']=pd.Categorical(df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog'], ["KP", "ARA",'External Source', 'Illegal Value'])
df2test =df2test.sort_values(['sort'])
df2test  = df2test.rename_axis(None)
cols = df2test.columns.tolist()
cols = [cols[-2]] + cols[:-2]
df2test = df2test[cols]
df2test['Query Type'] = 'Async'

df2test = df2test[['Query Type']+ list(df2test.columns[:-1])]

In [ ]:
df2test

<br>

## Converting the Pk's to hyperlink

<br>

In [ ]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [ ]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [ ]:
df.sort_index(inplace=True)

In [ ]:
df['Query Type'] = 'Async'

df = df[['Query Type']+ list(df.columns[:-1])]

In [ ]:
df

In [ ]:
#date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
#wks_name = 'Workflow Progress Tracker Asynchronous_' + date

In [ ]:
#wks_name

<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [ ]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *
import gspread_dataframe as gd

In [ ]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [ ]:
ws = gc.open("workflow_progress_tracker").worksheet("Workflow Progress Tracker_2021_10_01-08_34_02_AM")

#existing = gd.get_as_dataframe(ws)
existing = pd.DataFrame(ws.get_all_records())

existing.set_index('', inplace=True)



existing = existing.rename_axis(None)
existing.loc['pk'][:] = existing.loc['pk'][:].apply(lambda x: x.replace('" "', '","'))
existing.loc['pk'][1]

updated = existing.append(df)
gd.set_with_dataframe(ws, updated,include_index=True,include_column_header=True)

In [ ]:
#spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
#d2g.upload(df, spreadsheet_key, "TESTING ONLY", credentials=credentials, row_names=True)

In [ ]:
gc = gspread.service_account(filename='/Users/priyash/Documents/GitHub/minihackathons/Notebooks/araxworkflowprogresstesting-2632632db8be.json')
wksh = gc.open("workflow_progress_tracker")
#sh = wksh.worksheet(wks_name)
sh = wksh.worksheet("Workflow Progress Tracker_2021_10_01-08_34_02_AM")

In [ ]:
sh = wksh.worksheet("Workflow Progress Tracker_2021_10_01-08_34_02_AM")

In [ ]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(1,0,0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

In [ ]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.5, 0.0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

In [ ]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['No Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0.75, 0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

In [ ]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_CONTAINS', ['Error: 501']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.75, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

In [ ]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Unknown']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

In [ ]:
# Select a range
cell_list = sh.range('B19:AS19')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')


In [ ]:
# Select a range
cell_list = sh.range('B23:AS23')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')


In [ ]:
set_frozen(sh, cols=1, rows=1)

### Push Dataframe 2

<br>

In [ ]:
ws2 = gc.open("workflow_progress_tracker").worksheet("edge_attribute_source_2021_10_01-08_34_02_AM")

#existing = gd.get_as_dataframe(ws)
existing = pd.DataFrame(ws2.get_all_records())

existing.set_index('', inplace=True)



existing = existing.rename_axis(None)

updated = existing.append(df2test)
gd.set_with_dataframe(ws2, updated,include_index=True,include_column_header=True)


In [ ]:
set_frozen(ws2, cols=1, rows =1)